In [1]:
import random
import cupy as np
from data_process import get_CIFAR10_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax
from kaggle_submission import output_submission_csv
%matplotlib inline

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [2]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 40000
VAL_IMAGES = 10000

In [3]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))


Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [4]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [5]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on CIFAR

In [6]:
lr = 0.50
n_epochs = 10

percept_CIFAR = Perceptron(n_class_CIFAR, lr, n_epochs)
percept_CIFAR.train(X_train_CIFAR, y_train_CIFAR)


In [7]:
pred_percept = percept_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_CIFAR)))

The training accuracy is given by: 35.485000


### Validate Perceptron on CIFAR

In [8]:
pred_percept = percept_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_CIFAR)))

The validation accuracy is given by: 27.820000


### Test Perceptron on CIFAR

In [9]:
pred_percept = percept_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_CIFAR)))

The testing accuracy is given by: 28.170000


In [16]:
# HyperParameter tuning for Perceptron

lr_list = [0.00001, 0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.50, 0.75]
n_epoch_list = [10, 25, 100, 250, 500, 750, 1000]

best_perc_model = None
best_perc_acc = 0
best_params = None

param_list = []
acc_list = []
for temp_lr in lr_list: 
    for temp_n_epochs in n_epoch_list:
        temp_model = Perceptron(n_class_CIFAR, temp_lr, temp_n_epochs)
        temp_model.train(X_train_CIFAR, y_train_CIFAR)

        temp_pred = temp_model.predict(X_val_CIFAR)

        param_list.append((temp_lr, temp_n_epochs))
        acc_list = get_acc(temp_pred, y_val_CIFAR)


        if get_acc(temp_pred, y_val_CIFAR) > best_perc_acc:
            best_perc_model = temp_model
            best_perc_acc =  get_acc(temp_pred, y_val_CIFAR)
            best_params = (temp_lr, temp_n_epochs) 


# best_perc_index = np.argmax(np.array(acc_list))
print(f"best_accuracy - {best_perc_acc}, hyperparameters - {best_params}")

best_accuracy - 29.13, hyperparameters - (0.0001, 25)


### Perceptron_CIFAR Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 CIFAR. Use the following code to do so:

In [17]:
print(get_acc(best_perc_model.predict(X_val_CIFAR), y_val_CIFAR))

# Hard coded best model based on last cell results.
temp_model = Perceptron(n_class_CIFAR, 0.0001, 25)

output_submission_csv('kaggle/perceptron_submission_CIFAR.csv', temp_model.predict(X_test_CIFAR))

29.13


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on CIFAR

In [71]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

svm_CIFAR = SVM(n_class_CIFAR, lr, n_epochs, reg_const)
svm_CIFAR.train(X_train_CIFAR, y_train_CIFAR, batch_size = 250)

In [75]:
pred_svm = svm_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_CIFAR)))

The training accuracy is given by: 19.017500


### Validate SVM on CIFAR

In [76]:
pred_svm = svm_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_CIFAR)))

The validation accuracy is given by: 19.080000


### Test SVM on CIFAR

In [77]:
pred_svm = svm_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_CIFAR)))

The testing accuracy is given by: 18.890000


In [21]:
lr_list = [0.00001, 0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.50, 0.75]
n_epoch_list = [10, 25, 100, 250, 500, 750, 1000]
reg_const_list = [0.0001, 0.001, 0.05, 0.5, 0.75]
batch_size = [50, 250, 500, 1000, 5000]

best_svm_model = None
best_svm_acc = 0
best_svm_params = None

param_list = []
acc_list = []
for temp_lr in lr_list: 
    for temp_n_epochs in n_epoch_list:
        for temp_reg in reg_const_list:
            for batch in batch_size: 
                temp_model = SVM(n_class_CIFAR, temp_lr, temp_n_epochs, temp_reg)
                temp_model.train(X_train_CIFAR, y_train_CIFAR, batch_size = batch)
    
                temp_pred = temp_model.predict(X_val_CIFAR)
    
                param_list.append((temp_lr, temp_n_epochs, temp_reg, batch))
                acc_list = get_acc(temp_pred, y_val_CIFAR)
    
    
                if get_acc(temp_pred, y_val_CIFAR) > best_svm_acc:
                    best_svm_model = temp_model
                    best_svm_acc = get_acc(temp_pred, y_val_CIFAR)
                    best_svm_params = (temp_lr, temp_n_epochs, batch) 

print(f"best_accuracy - {best_svm_acc}, hyperparameters - {best_svm_params}")

### SVM_CIFAR Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 CIFAR. Use the following code to do so:

In [ ]:
output_submission_csv('kaggle/svm_submission_CIFAR.csv', svm_CIFAR.predict(X_test_CIFAR))

# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on CIFAR

In [65]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

softmax_CIFAR = Softmax(n_class_CIFAR, lr, n_epochs, reg_const)
softmax_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

0
1
2
3
4
5
6
7
8
9


/workspaces/linear_classifers_747/models/softmax.py:55: RuntimeWarning: divide by zero encountered in log
  


In [61]:
pred_softmax = softmax_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_CIFAR)))

The training accuracy is given by: 20.710000


### Validate Softmax on CIFAR

In [62]:
pred_softmax = softmax_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_CIFAR)))

The validation accuracy is given by: 20.840000


### Testing Softmax on CIFAR

In [63]:
pred_softmax = softmax_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_CIFAR)))

The testing accuracy is given by: 21.350000


In [ ]:
lr_list = [0.00001, 0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.50, 0.75]
n_epoch_list = [10, 25, 100, 250, 500, 750, 1000]
reg_const_list = [0.0001, 0.001, 0.05, 0.5, 0.75]
batch_size = [50, 250, 500, 1000, 5000]

best_smax_model = None
best_smax_acc = 0
best_smax_params = None

param_list = []
acc_list = []
for temp_lr in lr_list: 
    for temp_n_epochs in n_epoch_list:
        for temp_reg in reg_const_list:
            for batch in batch_size: 
                temp_model = Softmax(n_class_CIFAR, temp_lr, temp_n_epochs)
                temp_model.train(X_train_CIFAR, y_train_CIFAR, batch_size = batch)

                temp_pred = temp_model.predict(X_val_CIFAR)

                param_list.append((temp_lr, temp_n_epochs, batch))
                acc_list = get_acc(temp_pred, y_val_CIFAR)


                if get_acc(temp_pred, y_val_CIFAR) > best_smax_acc:
                    best_smax_model = temp_model
                    best_smax_acc = get_acc(temp_pred, y_val_CIFAR)
                    best_smax_params = (temp_lr, temp_n_epochs, batch) 

print(f"best_accuracy - {best_smax_acc}, hyperparameters - {best_smax_params}")

### Softmax_CIFAR Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 CIFAR. Use the following code to do so:

In [ ]:
output_submission_csv('kaggle/softmax_submission_CIFAR.csv', softmax_CIFAR.predict(X_test_CIFAR))